In [34]:
import numpy as np 
import pandas as pd 
import json
import re

from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

In [37]:
dataset = load_dataset("512duncanl/wh40k_novels")

Found cached dataset json (C:/Users/90530/.cache/huggingface/datasets/512duncanl___json/512duncanl--wh40k_novels-3308fde660ed265e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
df = pd.DataFrame(dataset['train'])
df.tail()

,text
4629,nce all directions were lost.\n\nThe years beg...
4630,tell what its final fate will be. From the so...
4631,me. I know it.'\n\n'You would do that?' Abadd...
4632,em the new cards he gathered from the burned c...
4633,inforce subframe structure at nine-six and nin...


In [118]:
def preprocess_data(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z0-9]', ' ', sentence)
    sentence = re.sub(' +', ' ', sentence)
    return sentence

cleaned_sentences = list(map(preprocess_data, list(df['text'].values)))
cleaned_sentences = cleaned_sentences
cleaned_sentences = ' '.join(cleaned_sentences)[:10000]

In [123]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts([cleaned_sentences])

tokenized_data = tokenizer.texts_to_sequences([cleaned_sentences])[0]

In [125]:
num_words = 3

input_data = []
output_data = []

for i in range(0, len(tokenized_data)-num_words):
    input_data.append(tokenized_data[i:i+num_words])
    output_data.append(tokenized_data[i+num_words])

input_data = np.array(input_data)
output_data = np.array(output_data)

In [133]:
print("input_data shape {}".format(input_data.shape))
print("output_data shape {}".format(output_data.shape))

input_data shape (1741, 3)
output_data shape (1741,)


In [13]:
model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=num_words))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))

model.add(Dense(len(tokenizer.word_index), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('nex_word_prediction.h5', monitor='loss', save_best_only=True, verbose=1)

model.fit(input_data, output_data, epochs=10, batch_size=64, callbacks=[checkpoint])